In [14]:
import pandas as pd
import json
import os

In [2]:
scraped_minutes_urls = pd.read_csv("scraped_minutes_urls.csv")

In [16]:
def get_filename(url):
    if url == "None":
        return None
    return f"{url.split('=')[-1]}.pdf"

In [20]:
scraped_minutes_urls["minutes_filename"] = scraped_minutes_urls["minues_pdf_url"].map(get_filename)
scraped_minutes_urls.set_index("id", inplace=True)

In [12]:
def get_fname(datetime_iso, meeting_type):
    """
    datetime_iso like: "2022-10-03T16:00:00" ",
    meeting_type like: "Accessibility Advisory Committee Meeting"
    """
    date = datetime_iso.split("T")[0]
    meeting_type = meeting_type.replace(" ", "-")
    meeting_type = meeting_type.replace(",","")
    return f"{date}.{meeting_type}"

def fill_minutes_fname(x):
    fname = f'{get_fname(x["datetime_iso"], x["meeting_type"])}.pdf'
    if not os.path.isfile(f"minutes_new/{fname}"):
        return None
    return fname

In [5]:
with open("generated_data/all_data_flat.json", "r") as f:
    all_data_flat = json.load(f)

In [6]:
all_data_flat_df = pd.DataFrame.from_records(all_data_flat)

In [16]:
all_data_flat_df["minutes_filename"] = all_data_flat_df.apply(fill_minutes_fname, axis=1)

In [56]:
# minutes_filename_dict = scraped_minutes_urls.to_dict()["minutes_filename"]
# all_data_flat_df["minutes_filename"] = all_data_flat_df["id"].map(minutes_filename_dict)

In [17]:
cols_of_interest = ["id", "meeting_type", "location", "datetime_iso", "agenda_url", "cancelled", "video_url", "minutes_filename"]

In [19]:
all_data_flat_df[cols_of_interest].to_json("generated_data/all_meeting_data.json", orient="records")

In [64]:
import scraping

In [80]:
meetings_url = "https://pub-peterborough.escribemeetings.com/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2022"

meetings_page = scraping.requests.get(meetings_url)
soup = scraping.BeautifulSoup(meetings_page.text, "html.parser")
# meeting_type_anchors = soup.find_all("a", {"class": "PastMeetingTypesName"})
# meeting_type_hrefs = [x.attrs["href"] for x in meeting_type_anchors]



In [81]:
meeting_type_anchors = soup.find_all("a", {"class": "PastMeetingTypesName"})

In [83]:
meeting_type_hrefs = [x.attrs["href"] for x in meeting_type_anchors]


In [84]:
meeting_type_hrefs

['https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2022&Expanded=Accessibility+Advisory+Committee+Meeting',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2022&Expanded=Ad+Hoc+Committee',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2022&Expanded=Airport+Strategic+Initiatives+Committee',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2022&Expanded=Arenas%2c+Parks+and+Recreation+Advisory+Committee+Meeting',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2022&Expanded=Arts%2c+Culture+and+Heritage+Advisory+Committee+Meeting',
 'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Year=2022&Expanded=Audit+Committee+Meetin

In [85]:
meeting_type_page = scraping.requests.post(meeting_type_hrefs[0])


In [97]:
meeting_page_text = meeting_type_page.text
soup = scraping.BeautifulSoup(meeting_page_text, "html.parser")
past_meeting_soup = soup.find("div", {"class": "past-meetings-region"})
if not past_meeting_soup:
    print("hi")
all_meetings_of_type_2021 = past_meeting_soup.find_all(
    "div", {"class": "meeting-header"}
)
parsed_meetings = []
for meeting_soup in all_meetings_of_type_2021:
    meeting = scraping.parse_meeting_soup(meeting_soup)
    parsed_meetings.append(meeting)

In [101]:
all_meetings_of_type_2021

[]

In [92]:
all_meetings_of_type_2021

[]

In [82]:
meeting_type_anchors[0]

<a aria-expanded="false" class="PastMeetingTypesName collapsed btn" data-target=".collapse0" data-toggle="collapse" href="https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&amp;wmode=transparent&amp;Year=2022&amp;Expanded=Accessibility+Advisory+Committee+Meeting" id="ctl00_MainContent_lvPastMeetingTypes_ctrl0_ExpandCollapseLink" role="button">
<span class="MeetingTypeNameText">Accessibility Advisory Committee Meeting </span>
<span class="MeetingTypeMeetingCount MeetingTypeNameText">(<span class="Year Year0">86</span><span class="Year Year2012">6</span><span class="Year Year2013">8</span><span class="Year Year2014">7</span><span class="Year Year2015">9</span><span class="Year Year2016">9</span><span class="Year Year2017">10</span><span class="Year Year2018">7</span><span class="Year Year2019">7</span><span class="Year Year2020">7</span><span class="Year Year2021">8</span><span class="Year Year2022">8</span>)
                                           

In [75]:
parsed_meetings: scraping.List[scraping.Meeting] = []
for n, href in enumerate(meeting_type_hrefs[:]):
    print(f"{n} of {len(meeting_type_hrefs)}")
    meeting_type_page = scraping.requests.post(href)
    parsed_meetings_of_type = scraping.parse_meeting_page_text(meeting_type_page.text)
    parsed_meetings += parsed_meetings_of_type
    with open(f"data2/part_{n}.json", "w") as f:
        scraping.json.dump(parsed_meetings_of_type, f)

0 of 32
1 of 32
2 of 32



KeyboardInterrupt



In [78]:
href

'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Expanded=Airport+Strategic+Initiatives+Committee'

In [79]:
abc = "https://pub-peterborough.escribemeetings.com/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Expanded=Airport+Strategic+Initiatives+Committee"

In [76]:
meeting_type_page

<Response [200]>

In [102]:
meeting_type_page

<Response [200]>

In [103]:
href

'https://pub-peterborough.escribemeetings.com:443/meetingscalendarview.aspx?FillWidth=1&wmode=transparent&Expanded=Airport+Strategic+Initiatives+Committee'

In [104]:
abc = scraping.requests.post(href)

In [106]:
text = abc.text

In [107]:
with open("test.html", "w") as f:
    f.write(text)
    f.close()